In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [3]:
data = spark.read.table('customerchurn')

In [4]:
data.printSchema()

In [5]:
data.columns

In [6]:
from pyspark.ml.feature import VectorAssembler

In [7]:
assembler = VectorAssembler(inputCols = ['Age','Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol = 'features')

In [8]:
output = assembler.transform(data)

In [9]:
final_data = output.select('features','Churn')

In [10]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

In [11]:
from pyspark.ml.classification import LogisticRegression

In [12]:
lr_churn = LogisticRegression(labelCol='Churn')

In [13]:
final_data.printSchema()

In [14]:
fitted_churn_model = lr_churn.fit(train_churn)

In [15]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [16]:
pred_and_label = fitted_churn_model.evaluate(test_churn)

In [17]:
pred_and_label.predictions.show()

In [18]:
### PREDICTING ON ALL DATA

In [19]:
final_lr_model = lr_churn.fit(final_data)

In [20]:
new_customers =spark.read.table('new_customers')

In [21]:
test_new_customer = assembler.transform(new_customers)

In [22]:
Final_results = final_lr_model.transform(test_new_customer)

In [23]:
Final_results.select('Names','Company','prediction').show()